# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [1]:
! pip install -q schedule pytest # установка библиотек, если ещё не

In [2]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import os
import re
import time
import json

import requests
import schedule
from bs4 import BeautifulSoup
from tqdm import tqdm

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [3]:
def get_book_data(book_url: str) -> dict:
    """
    Извлекает данные о книге с сайта Books to Scrape.
    
    Парсит информацию о книге включая название, рейтинг, цену, наличие и другие характеристики из таблицы продукта.
    
    Args:
        book_url (str): URL страницы книги на сайте books.toscrape.com
        
    Returns:
        dict: Словарь с данными о книге в формате:
            {
                'name': 'Название книги',
                'rating': '5',  # от 1 до 5
                'Price': '£51.77',
                'Availability': '22',
                'UPC': 'a897fe39b1053632',
                ...
            }
        None: в случае ошибки запроса или парсинга
        
    Raises:
        requests.RequestException: при ошибках сетевого запроса
        Exception: при ошибках парсинга HTML
    """
    try:
        # HTTP-запрос и базовая валидация
        response = requests.get(book_url, timeout=10)
        response.encoding = 'utf-8'
        response.raise_for_status()
        
        # Инициализация парсера и словаря
        soup = BeautifulSoup(response.text, "html.parser")
        item = {}
        
        # Парсинг названия книги
        product_main = soup.find("div", class_="col-sm-6 product_main")
        if product_main:
            h1_element = product_main.find("h1")
            if h1_element:
                item['name'] = h1_element.get_text(strip=True)
            else:
                item['name'] = "Название не найдено"
        else:
            item['name'] = "Блок продукта не найден"
        
        # Парсинг рейтинга
        rating_element = soup.find('p', class_='star-rating')
        if rating_element:
            classes_str = ' '.join(rating_element.get('class', []))
            rating_match = re.findall(r'One|Two|Three|Four|Five', classes_str)
            if rating_match:
                rating_map = {'One': '1', 'Two': '2', 'Three': '3', 'Four': '4', 'Five': '5'}
                item['rating'] = rating_map.get(rating_match[0], 'Unknown')
            else:
                item['rating'] = "Рейтинг не распознан"
        else:
            item['rating'] = "Элемент рейтинга не найден"
        
        # Табличные данные
        table = soup.find('table', class_='table table-striped')
        if table:
            rows = table.find_all('tr')
            for row in rows:
                headers = [x.get_text(' ', strip=True) for x in row.find_all('th')]
                values = [x.get_text(' ', strip=True) for x in row.find_all('td')]
                
                if headers and values:
                    key = headers[0]
                    value = values[0]
                    item[key] = value
            
            # Обработка числа книг в наличии
            if "Availability" in item:
                numbers = re.findall(r'\d+', item["Availability"])
                item["Availability"] = numbers[0] if numbers else "0"
        else:
            print("Таблица не найдена")
     
        return item
       
    except requests.RequestException as e:
        print(f"Ошибка запроса: {e}")
        return None
    
    except Exception as e: 
        print(f"Ошибка парсинга: {e}")
        return None

In [4]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'name': 'A Light in the Attic',
 'rating': '3',
 'UPC': 'a897fe39b1053632',
 'Product Type': 'Books',
 'Price (excl. tax)': '£51.77',
 'Price (incl. tax)': '£51.77',
 'Tax': '£0.00',
 'Availability': '22',
 'Number of reviews': '0'}

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [5]:
def scrape_books(save_to_file: bool = False) -> list:
    """
    Парсит все книги с сайта Books to Scrape.
    
    Args:
        save_to_file: Если True, сохраняет данные в JSON файл
        
    Returns:
        List[dict]: Список словарей с данными о книгах
    """
    start_time = time.time()
    all_books_data = []
    page_number = 1
    
    while True:
        try:
            url = f'http://books.toscrape.com/catalogue/page-{page_number}.html'
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            
            if response.status_code == 404:
                break
                
            soup = BeautifulSoup(response.text, "html.parser")
            books = soup.find_all('article', class_='product_pod')
            all_books_href = []
            
            for book in books:
                all_books_href.append(book.find('h3').find('a')['href'])
                
            for href in tqdm(all_books_href, desc=f"Страница {page_number}", leave=False):
                try:
                    books_url = f'http://books.toscrape.com/catalogue/{href}'
                    book_data = get_book_data(books_url)
                    if book_data:
                        all_books_data.append(book_data)
                except Exception as e:  # pylint: disable=broad-exception-caught
                    print(f"Ошибка парсинга книги {href}: {e}")
                    continue
                    
            page_number += 1
            
        except requests.HTTPError as e:
            if e.response.status_code == 404:
                break
            print(f"HTTP ошибка на странице {page_number}: {e}")
            page_number += 1
        except requests.RequestException as e:
            print(f"Ошибка запроса страницы {page_number}: {e}")
            page_number += 1
        except Exception as e:  # pylint: disable=broad-exception-caught
            print(f"Ошибка на странице {page_number}: {e}")
            page_number += 1
    
    total_time = time.time() - start_time
    minutes = int(total_time // 60)
    seconds = int(total_time % 60)
    
    print(f"Общее время: {minutes} мин {seconds} сек")
            
    if save_to_file:
        try:
            with open("books_data.json", "w", encoding='utf-8') as f:
                json.dump(all_books_data, f, indent=2, ensure_ascii=False)
            print("Данные сохранены в books_data.json")
        except Exception as e:  # pylint: disable=broad-exception-caught
            print(f"Ошибка сохранения файла: {e}")
                
    return all_books_data

In [6]:
# Проверка работоспособности функции
res = scrape_books(save_to_file=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

Общее время: 8 мин 17 сек
Данные сохранены в books_data.json
<class 'list'> 1000


## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [7]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
schedule.every().day.at('19:00').do(scrape_books,save_to_file=True)

while True:
    schedule.run_pending()
    time.sleep(60)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

KeyboardInterrupt: 

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [8]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
os.chdir('..')
! pytest test/test_scraper.py

]9;4;3;\============================= test session starts ==============================
platform linux -- Python 3.12.3, pytest-9.0.0, pluggy-1.6.0
rootdir: /home/ivan/books_scraper
plugins: anyio-4.11.0, cov-7.0.0
collected 4 items                                                              

test/test_scraper.py ]9;4;1;0\.]9;4;1;25\.]9;4;1;50\.]9;4;1;75\.                                                [100%]]9;4;0;\

============================== 4 passed in 2.36s ===============================


## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```